In [478]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from functools import reduce
import operator
from pymongo import MongoClient
from pymongo import GEOSPHERE
import sys
sys.path.append('../')
from bson.json_util import dumps
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

import src.functions as func

### Choosing 3 cities to place my office:

#### A. Madrid:

In [2]:
madrid=func.geocode("Castellana 89, Paseo de la Castellana, Madrid") #Torre Picasso.

In [3]:
madrid

{'type': 'Point', 'coordinates': ['40.44980', '-3.69164']}

#### B. Málaga:

In [6]:
malaga=func.geocode("Calle Larios, Malaga")

In [7]:
malaga

{'type': 'Point', 'coordinates': ['36.72048', '-4.42166']}

#### C. Barcelona:

In [8]:
barcelona=func.geocode("Plaça de Catalunya, Barcelona")

In [9]:
barcelona

{'type': 'Point', 'coordinates': ['41.38781', '2.16920']}

### Prioritizing some requirements over othes:

- **Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.**
- 30% of the company staff have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- **Executives like Starbucks A LOT. Ensure there's a starbucks not too far.**
- **Account managers need to travel a lot.**
- **Everyone in the company is between 25 and 40, give them some place to go party.**
- **The CEO is vegan.**
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

### API Foursquare requests:

In [10]:
load_dotenv()

True

In [11]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [12]:
url = "https://favqs.com/api/session"
url_query = 'https://api.foursquare.com/v2/venues/search'

### A. MADRID.

#### A. Madrid-Empresas de diseño:

In [365]:
parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "empresas de diseño"
}

In [366]:
dis_mad = requests.get(url_query, params = parametros).json()

In [367]:
diseños_md = dis_mad["response"]["venues"]

In [479]:
lista_diseñomad=func.extraetodo(diseños_md)

In [480]:
datos_diseñomadrid = pd.DataFrame(lista_diseñomad)

In [481]:
diseño_madlimpio=datos_diseñomadrid.drop([0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,17,18,19,20,21,23,24,25,28,29],axis=0)

In [482]:
diseño_madlimpio["categoria"] = "Diseño"
diseño_madlimpio.head()

,nombre,latitud,longitud,location,categoria
7,Escuela Superior De Diseño,40.456288,-3.712720,"{'type': 'Point', 'coordinates': [40.456287639...",Diseño
16,Estudio de diseño Juan Gatti,40.423439,-3.686265,"{'type': 'Point', 'coordinates': [40.423439, -...",Diseño
22,Flor Canela. Tienda De Diseño. Mercado Fuencarral,40.422942,-3.699797,"{'type': 'Point', 'coordinates': [40.422942, -...",Diseño
26,Vivero de Empresas de San Blas. Madrid Emprende,40.446471,-3.612300,"{'type': 'Point', 'coordinates': [40.446470528...",Diseño
27,altura x estudio de diseño,40.409821,-3.705907,"{'type': 'Point', 'coordinates': [40.409820698...",Diseño


#### A. Madrid-Starbucks:

In [373]:
parametros_St = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Starbucks"
}

In [374]:
St_mad = requests.get(url_query, params = parametros_St).json()

In [375]:
St_datamd = St_mad["response"]["venues"]

In [376]:
lista_starbucksmad=func.extraetodo(St_datamd)

In [377]:
datos_Starbucksmadrid=pd.DataFrame(lista_starbucksmad)

In [378]:
datos_Starbucksmadrid["categoria"] = "Starbucks"
datos_Starbucksmadrid.head()

,nombre,latitud,longitud,location,categoria
0,Starbucks,40.447489,-3.693093,"{'type': 'Point', 'coordinates': [40.447488723...",Starbucks
1,Starbucks,40.447729,-3.695175,"{'type': 'Point', 'coordinates': [40.447728697...",Starbucks
2,Starbucks,40.455046,-3.693885,"{'type': 'Point', 'coordinates': [40.455046, -...",Starbucks
3,Starbucks,40.432968,-3.683220,"{'type': 'Point', 'coordinates': [40.432967606...",Starbucks
4,Starbucks,40.433564,-3.686829,"{'type': 'Point', 'coordinates': [40.433563917...",Starbucks


#### A. Madrid-Airports:

In [379]:
parametros_A = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Airport"
}

In [380]:
airport_mad=requests.get(url_query, params=parametros_A).json()

In [381]:
air_datamd = airport_mad["response"]["venues"]

In [382]:
lista_airportmad=func.extraetodo(air_datamd)

In [383]:
datos_airportmadrid=pd.DataFrame(lista_airportmad)

In [384]:
aeropuerto_madrid=datos_airportmadrid.drop([0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21,22,24,25,26,27,28,29])

In [385]:
aeropuerto_madrid

,nombre,latitud,longitud,location
4,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162..."
17,#NASF AIRPORT,40.464236,-3.569769,"{'type': 'Point', 'coordinates': [40.464236, -..."
23,Parada Atocha Renfe - Línea Exprés Aeropuerto,40.406724,-3.689763,"{'type': 'Point', 'coordinates': [40.406724103..."


In [386]:
aeropuerto_madrid["categoria"] = "aeropuerto"
aeropuerto_madrid.head()

,nombre,latitud,longitud,location,categoria
4,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162...",aeropuerto
17,#NASF AIRPORT,40.464236,-3.569769,"{'type': 'Point', 'coordinates': [40.464236, -...",aeropuerto
23,Parada Atocha Renfe - Línea Exprés Aeropuerto,40.406724,-3.689763,"{'type': 'Point', 'coordinates': [40.406724103...",aeropuerto


#### A. Madrid-Disco:

In [387]:
parametros_D = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Disco"
}

In [388]:
disco_mad=requests.get(url_query,params=parametros_D).json()

In [389]:
disco_datamad=disco_mad["response"]["venues"]

In [390]:
lista_discomad=func.extraetodo(disco_datamad)

In [391]:
datos_discomadrid=pd.DataFrame(lista_discomad)

In [392]:
datos_discomadrid["categoria"] = "Disco"
datos_discomadrid.head()

,nombre,latitud,longitud,location,categoria
0,Disco Rug & Play,40.451888,-3.696266,"{'type': 'Point', 'coordinates': [40.451888, -...",Disco
1,Panamericana Disco,40.452806,-3.690264,"{'type': 'Point', 'coordinates': [40.452806048...",Disco
2,Disco Yedra Pub,40.434041,-3.714618,"{'type': 'Point', 'coordinates': [40.434040912...",Disco
3,Disco Express,40.434539,-3.710257,"{'type': 'Point', 'coordinates': [40.434538914...",Disco
4,Destellos Pub Disco,40.437682,-3.709905,"{'type': 'Point', 'coordinates': [40.437681716...",Disco


####  A. Madrid-Vegan:

In [393]:
parametros_V= {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Vegan"
}

In [394]:
vegan_mad=requests.get(url_query,params=parametros_V).json()

In [395]:
vegan_datamad=vegan_mad["response"]["venues"]

In [396]:
lista_veganmad=func.extraetodo(vegan_datamad)

In [397]:
datos_veganmadrid=pd.DataFrame(lista_veganmad)

In [398]:
vegan_madridlimpio=datos_veganmadrid.drop([15,29])

In [399]:
vegan_madridlimpio["categoria"] = "Vegan"
vegan_madridlimpio.head()

,nombre,latitud,longitud,location,categoria
0,Vegan Streets Madrid Castellana,40.442456,-3.679973,"{'type': 'Point', 'coordinates': [40.4424564, ...",Vegan
1,Vegan Streets Madrid Centro,40.417512,-3.697636,"{'type': 'Point', 'coordinates': [40.4175117, ...",Vegan
2,vegetarian / vegan restaurant,40.432974,-3.686693,"{'type': 'Point', 'coordinates': [40.432974, -...",Vegan
3,Mad Mad Vegan,40.424183,-3.700731,"{'type': 'Point', 'coordinates': [40.424183, -...",Vegan
4,Los Andenes Vegan,40.435206,-3.710348,"{'type': 'Point', 'coordinates': [40.435206, -...",Vegan


#### Suming up all requirements in a single list (Madrid).

In [400]:
MADRID_df=func.concat_requirements(diseño_madlimpio, datos_Starbucksmadrid, aeropuerto_madrid, datos_discomadrid, vegan_madridlimpio)

In [401]:
MADRID_df["ciudad"] = "Madrid"

In [402]:
MADRID_df.shape

(96, 6)

In [496]:
MADRID_df.sample()

,nombre,latitud,longitud,location,categoria,ciudad
25,Dolce & Vegana,40.413803,-3.717875,"{'type': 'Point', 'coordinates': [40.413803, -...",Vegan,Madrid


In [578]:
MADRID_df.to_csv("../data/MADRID.csv")

In [497]:
lista_madrid = MADRID_df.to_dict(orient = "records")

#### B. Málaga-Empresas de diseño:

In [297]:
parametros_mlg = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{malaga['coordinates'][0]}, {malaga['coordinates'][1]}",
    "query": "empresas de diseño"
}

In [298]:
dis_mlg=requests.get(url_query,params=parametros_mlg).json()

In [299]:
diseños_mlg = dis_mlg["response"]["venues"]

In [300]:
lista_diseñomlg=func.extraetodo(diseños_mlg)

In [301]:
datos_diseñomlg=pd.DataFrame(lista_diseñomlg)

In [302]:
diseño_mlglimpio=datos_diseñomlg.drop([3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19])

In [303]:
diseño_mlglimpio

,nombre,latitud,longitud,location
0,Empresas de reformas,36.721033,-4.410110,"{'type': 'Point', 'coordinates': [36.721033086..."
1,Tango Estudio de diseño,36.721188,-4.424591,"{'type': 'Point', 'coordinates': [36.721187882..."
2,Medio Melón Estudio de Diseño Gráfico,36.723841,-4.418049,"{'type': 'Point', 'coordinates': [36.723840858..."
8,Gauss Multimedia Escuela de Diseño,36.720976,-4.474269,"{'type': 'Point', 'coordinates': [36.720975698..."


In [304]:
diseño_mlglimpio["categoria"] = "Diseño"
diseño_mlglimpio.head()

,nombre,latitud,longitud,location,categoria
0,Empresas de reformas,36.721033,-4.410110,"{'type': 'Point', 'coordinates': [36.721033086...",Diseño
1,Tango Estudio de diseño,36.721188,-4.424591,"{'type': 'Point', 'coordinates': [36.721187882...",Diseño
2,Medio Melón Estudio de Diseño Gráfico,36.723841,-4.418049,"{'type': 'Point', 'coordinates': [36.723840858...",Diseño
8,Gauss Multimedia Escuela de Diseño,36.720976,-4.474269,"{'type': 'Point', 'coordinates': [36.720975698...",Diseño


#### B. Málaga-Aeropuerto:

In [305]:
parametros_mlg_air = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{malaga['coordinates'][0]}, {malaga['coordinates'][1]}",
    "query": "airport"
}

In [306]:
air_mlg=requests.get(url_query,params=parametros_mlg_air).json()

In [307]:
airport_mlg = air_mlg["response"]["venues"]

In [308]:
lista_airportmlg=func.extraetodo(airport_mlg)

In [309]:
datos_airmlg=pd.DataFrame(lista_airportmlg)

In [310]:
aeropuerto_mlglimpio=datos_airmlg.iloc[2:3]

In [311]:
aeropuerto_mlglimpio

,nombre,latitud,longitud,location
2,Aeropuerto de Málaga - Costa del Sol (AGP) (Ae...,36.677056,-4.492091,"{'type': 'Point', 'coordinates': [36.677056143..."


In [483]:
aeropuerto_mlglimpio["categoria"] = "Aeropuerto"
aeropuerto_mlglimpio

<ipython-input-483-8070b665eb6f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aeropuerto_mlglimpio["categoria"] = "Aeropuerto"


,nombre,latitud,longitud,location,categoria
2,Aeropuerto de Málaga - Costa del Sol (AGP) (Ae...,36.677056,-4.492091,"{'type': 'Point', 'coordinates': [36.677056143...",Aeropuerto


#### B. Málaga-Starbucks:

In [313]:
parametros_mlg_st = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{malaga['coordinates'][0]}, {malaga['coordinates'][1]}",
    "query": "Starbucks"
}

In [314]:
st_mlg=requests.get(url_query,params=parametros_mlg_st).json()

In [315]:
Starbucks_mlg = st_mlg["response"]["venues"]

In [316]:
lista_Starbucks_mlg=func.extraetodo(Starbucks_mlg)

In [317]:
starbucks_dfmlg=pd.DataFrame(lista_Starbucks_mlg)

In [485]:
starbucks_dfmlg["categoria"] = "Starbucks"
starbucks_dfmlg.sample()

,nombre,latitud,longitud,location,categoria
8,Starbucks On The Go,36.613181,-4.52859,"{'type': 'Point', 'coordinates': [36.6131812, ...",Starbucks


#### B. Málaga-Disco:

In [319]:
parametros_mlg_disco = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{malaga['coordinates'][0]}, {malaga['coordinates'][1]}",
    "query": "disco"
}

In [320]:
disco_mlg=requests.get(url_query,params=parametros_mlg_disco).json()

In [321]:
disco_mlg = disco_mlg["response"]["venues"]

In [322]:
lista_disco_mlg=func.extraetodo(disco_mlg)

In [323]:
disco_dfmlg=pd.DataFrame(lista_disco_mlg)

In [324]:
disco_mlglimpio=disco_dfmlg.drop([4,6,9,12])

In [487]:
disco_mlglimpio["categoria"] = "Disco"
disco_mlglimpio.sample()

,nombre,latitud,longitud,location,categoria
22,Disco Pub Yuka,36.766087,-4.709443,"{'type': 'Point', 'coordinates': [36.766086570...",Disco


#### B. Málaga-Vegan:

In [326]:
parametros_mlg_vegan = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{malaga['coordinates'][0]}, {malaga['coordinates'][1]}",
    "query": "Vegan"
}

In [327]:
vegan_mlg=requests.get(url_query,params=parametros_mlg_vegan).json()

In [328]:
vegan_mlg = vegan_mlg["response"]["venues"]

In [329]:
lista_vegan_mlg=func.extraetodo(vegan_mlg)

In [330]:
vegan_mlgdf=pd.DataFrame(lista_vegan_mlg)

In [489]:
vegan_mlgdf["categoria"] = "Vegan"
vegan_mlgdf.sample()

,nombre,latitud,longitud,location,categoria
9,My Lola Vegan Food,36.540123,-4.627398,"{'type': 'Point', 'coordinates': [36.540123, -...",Vegan


#### Suming up all requirements in a single list (Málaga).

In [490]:
MALAGA_df=func.concat_requirements(diseño_mlglimpio, aeropuerto_mlglimpio, starbucks_dfmlg, disco_mlglimpio, vegan_mlgdf)

In [491]:
MALAGA_df["ciudad"] = "Málaga"

In [579]:
MALAGA_df.to_csv("../data/MALAGA.csv")

In [493]:
lista_malaga = MALAGA_df.to_dict(orient = "records")

#### C. Barcelona-Design:

In [337]:
parametros_bcn_dis = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{barcelona['coordinates'][0]}, {barcelona['coordinates'][1]}",
    "query": "Diseño"
}

In [338]:
dis_bcn=requests.get(url_query,params=parametros_bcn_dis).json()

diseño_bcn = dis_bcn["response"]["venues"]

lista_diseñobcn=func.extraetodo(diseño_bcn)
diseño_bcndf=pd.DataFrame(lista_diseñobcn)

In [339]:
diseño_bcndf=diseño_bcndf.drop([1,2,5,11,18,19,21,22])

In [340]:
diseño_bcndf["categoria"] = "Diseño"
diseño_bcndf.head()

,nombre,latitud,longitud,location,categoria
0,Tuctucbarcelona —Diseño Gráfico y Web—,41.386273,2.174185,"{'type': 'Point', 'coordinates': [41.386273475...",Diseño
3,Diseño Paginas Web Yá Barcelona,41.380311,2.169969,"{'type': 'Point', 'coordinates': [41.380311, 2...",Diseño
4,"RC2 Ingenieria y Diseño, S.L",41.390638,2.174306,"{'type': 'Point', 'coordinates': [41.390638180...",Diseño
6,Dulce Diseño El Born,41.388119,2.178501,"{'type': 'Point', 'coordinates': [41.388118801...",Diseño
7,Socialweb Diseño Web Barcelona,41.377837,2.129298,"{'type': 'Point', 'coordinates': [41.377837033...",Diseño


#### C. Barcelona-Airport:

In [341]:
parametros_bcn_air = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{barcelona['coordinates'][0]}, {barcelona['coordinates'][1]}",
    "query": "Airport"
}

In [342]:
air_bcn=requests.get(url_query,params=parametros_bcn_air).json()

airport_bcn = air_bcn["response"]["venues"]

lista_airportbcn=func.extraetodo(airport_bcn)

In [343]:
air_bcndf=pd.DataFrame(lista_airportbcn)

In [344]:
air_bcnlimpio=air_bcndf.head(1)

In [345]:
air_bcnlimpio["categoria"] = "Aeropuerto"
air_bcnlimpio.head()

<ipython-input-345-1823c9e4b61a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  air_bcnlimpio["categoria"] = "Aeropuerto"


,nombre,latitud,longitud,location,categoria
0,Barcelona–El Prat Josep Tarradellas Airport (B...,41.295607,2.076223,"{'type': 'Point', 'coordinates': [41.295606993...",Aeropuerto


#### C. Barcelona-Starbucks:

In [346]:
parametros_bcn_st = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{barcelona['coordinates'][0]}, {barcelona['coordinates'][1]}",
    "query": "Starbucks"
}

In [347]:
st_bcn=requests.get(url_query,params=parametros_bcn_st).json()

starbucks_bcn = st_bcn["response"]["venues"]

lista_starbucksbcn=func.extraetodo(starbucks_bcn)

In [348]:
starbucks_dfbcn=pd.DataFrame(lista_starbucksbcn)

In [349]:
starbucks_dfbcn["categoria"] = "Starbucks"
starbucks_dfbcn.head()

,nombre,latitud,longitud,location,categoria
0,Starbucks Rambla Cataluña,41.387954,2.167901,"{'type': 'Point', 'coordinates': [41.387953542...",Starbucks
1,Starbucks Pelai,41.385805,2.167858,"{'type': 'Point', 'coordinates': [41.385804589...",Starbucks
2,Starbucks Rambla Canaletas,41.383485,2.171382,"{'type': 'Point', 'coordinates': [41.383485318...",Starbucks
3,Starbucks Plaza Universitat,41.386418,2.165150,"{'type': 'Point', 'coordinates': [41.386417656...",Starbucks
4,Starbucks Sant Pere,41.388783,2.172063,"{'type': 'Point', 'coordinates': [41.388782617...",Starbucks


#### C. Barcelona-Disco:

In [350]:
parametros_bcn_disco = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{barcelona['coordinates'][0]}, {barcelona['coordinates'][1]}",
    "query": "Disco"
}

In [351]:
disco_bcn=requests.get(url_query,params=parametros_bcn_disco).json()

discoteca_bcn = disco_bcn["response"]["venues"]

lista_discotecabcn=func.extraetodo(discoteca_bcn)

In [352]:
disco_bcndf=pd.DataFrame(lista_discotecabcn)

In [353]:
disco_bcnlimpio=disco_bcndf.drop([8,21,22])

In [354]:
disco_bcnlimpio["categoria"] = "Disco"
disco_bcnlimpio.head()

,nombre,latitud,longitud,location,categoria
0,Dirty House Disco,41.387745,2.168554,"{'type': 'Point', 'coordinates': [41.387745, 2...",Disco
1,Discos Paradiso,41.382223,2.165922,"{'type': 'Point', 'coordinates': [41.382223232...",Disco
2,Disco Bus,41.389916,2.171634,"{'type': 'Point', 'coordinates': [41.389916, 2...",Disco
3,Disco Pub Joe's,41.377008,2.137311,"{'type': 'Point', 'coordinates': [41.377008007...",Disco
4,Discoteca Hotel Omm,41.396143,2.160937,"{'type': 'Point', 'coordinates': [41.396142977...",Disco


#### C. Barcelona-Vegan:

In [355]:
parametros_bcn_veg = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{barcelona['coordinates'][0]}, {barcelona['coordinates'][1]}",
    "query": "Vegan"
}

In [356]:
veg_bcn=requests.get(url_query,params=parametros_bcn_veg).json()

vegan_bcn = veg_bcn["response"]["venues"]

lista_veganbcn=func.extraetodo(vegan_bcn)

In [357]:
vegan_bcndf=pd.DataFrame(lista_veganbcn)

In [358]:
vegan_bcnlimpio=vegan_bcndf.drop([2,4,15])

In [359]:
vegan_bcnlimpio["categoria"] = "Vegan"
vegan_bcnlimpio.head()

,nombre,latitud,longitud,location,categoria
0,Vegan Junk Food Bar,41.383386,2.180255,"{'type': 'Point', 'coordinates': [41.383385646...",Vegan
1,All Vegan,41.382820,2.169548,"{'type': 'Point', 'coordinates': [41.38282, 2....",Vegan
3,ENJOY VEGAN,41.388152,2.180673,"{'type': 'Point', 'coordinates': [41.3881518, ...",Vegan
5,Monchito Vegan,41.381030,2.179852,"{'type': 'Point', 'coordinates': [41.38103, 2....",Vegan
6,Monchito Vegan,41.380971,2.179880,"{'type': 'Point', 'coordinates': [41.380970639...",Vegan


#### Suming up all requirements in a single list (Barcelona).

In [360]:
BARCELONA_df=func.concat_requirements(diseño_bcndf, air_bcnlimpio, starbucks_dfbcn, disco_bcnlimpio, vegan_bcnlimpio)


In [361]:
BARCELONA_df["ciudad"] = "Barcelona"

In [580]:
BARCELONA_df.to_csv("../data/BARCELONA.csv")

In [581]:
lista_bcn = BARCELONA_df.to_dict(orient = "records")

### Exporting data to Mongo DB Compass.

In [494]:
client = MongoClient("localhost:27017")
db = client.get_database("PROYECTO_3")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROYECTO_3')

#### A. Madrid:

In [498]:
coleccion1 = db["MADRID"]
coleccion1.insert_many(lista_madrid)

#### B. Málaga:

In [499]:
coleccion2 = db["MALAGA"]
coleccion2.insert_many(lista_malaga)

#### C. Barcelona:

In [407]:
coleccion3 = db["BARCELONA"]
coleccion3.insert_many(lista_bcn)

### Geoqueries:

#### A. Madrid:

In [408]:
MADRID = db.get_collection("MADRID") # Me conecto a la colección "MADRID".

In [409]:
MADRID

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROYECTO_3'), 'MADRID')

In [410]:
db.MADRID.create_index([("location", GEOSPHERE)]) # Creo el index 2dsphere.

'location_2dsphere'

In [411]:
query=[{
"$geoNear": {'near': [40.4498,-3.69164 ],
             'distanceField': 'distance',
             'maxDistance': 2000,
             'distanceMultiplier': 6371,
             'spherical'  : True}}]

In [412]:
geoloc=MADRID.aggregate(query)

In [413]:
geoquery_madrid=json.loads(dumps(geoloc))

In [414]:
df_madrid = pd.DataFrame(geoquery_madrid)
type(df_madrid)
df_madrid.drop_duplicates(["latitud","longitud"],inplace=True)

In [415]:
df_madrid.head()

,_id,nombre,latitud,longitud,location,categoria,ciudad,distance
0,{'$oid': '6192843bf7bf3a2a12a2b49c'},Starbucks,40.447489,-3.693093,"{'type': 'Point', 'coordinates': [40.447488723...",Starbucks,Madrid,0.303146
9,{'$oid': '6192e6c17c6b9ec9dabfa339'},Panamericana Disco,40.452806,-3.690264,"{'type': 'Point', 'coordinates': [40.452806048...",Disco,Madrid,0.366997
18,{'$oid': '6192843bf7bf3a2a12a2b4cb'},la cubana discoteca,40.450147,-3.695196,"{'type': 'Point', 'coordinates': [40.450147, -...",Disco,Madrid,0.397279
27,{'$oid': '6192a1801f83f52cc2012b55'},Starbucks,40.447729,-3.695175,"{'type': 'Point', 'coordinates': [40.447728697...",Starbucks,Madrid,0.455384
36,{'$oid': '6192ad49103aea2c0e601089'},Disco Rug & Play,40.451888,-3.696266,"{'type': 'Point', 'coordinates': [40.451888, -...",Disco,Madrid,0.564160


In [416]:
madrid_agrupado=df_madrid.groupby(["categoria"])["distance"].mean()

In [417]:
dfMADRID=pd.DataFrame(madrid_agrupado).reset_index()

In [418]:
dfMADRID

,categoria,distance
0,Disco,3.478146
1,Diseño,3.664390
2,Starbucks,3.912457
3,Vegan,4.920965
4,aeropuerto,10.723970


In [419]:
func.normalize(dfMADRID,"distance")

[0.0, 0.025703623845056835, 0.05993946234668505, 0.19912424208109353, 1.0]

In [420]:
dfMADRID["normalizado"]=func.normalize(dfMADRID,"distance")

In [421]:
dfMADRID

,categoria,distance,normalizado
0,Disco,3.478146,0.000000
1,Diseño,3.664390,0.025704
2,Starbucks,3.912457,0.059939
3,Vegan,4.920965,0.199124
4,aeropuerto,10.723970,1.000000


In [422]:
dfMADRID["weights"] = func.weights(dfMADRID)

In [423]:
dfMADRID

,categoria,distance,normalizado,weights
0,Disco,3.478146,0.000000,0.000000
1,Diseño,3.664390,0.025704,0.005141
2,Starbucks,3.912457,0.059939,0.023976
3,Vegan,4.920965,0.199124,0.019912
4,aeropuerto,10.723970,1.000000,0.100000


In [424]:
dfMADRID["ciudad"] = "Madrid"

In [425]:
dfMADRID

,categoria,distance,normalizado,weights,ciudad
0,Disco,3.478146,0.000000,0.000000,Madrid
1,Diseño,3.664390,0.025704,0.005141,Madrid
2,Starbucks,3.912457,0.059939,0.023976,Madrid
3,Vegan,4.920965,0.199124,0.019912,Madrid
4,aeropuerto,10.723970,1.000000,0.100000,Madrid


#### B. Málaga:

In [500]:
MALAGA = db.get_collection("MALAGA") # Me conecto a la colección "MALAGA".

MALAGA

In [501]:
db.MALAGA.create_index([("location", GEOSPHERE)]) # Creo el index 2dsphere.

'location_2dsphere'

In [510]:
query_mlg=[{
"$geoNear": {'near': [36.72048, -4.42166],
             'distanceField': 'distance',
             'maxDistance': 2000,
             'distanceMultiplier': 6371,
             'spherical':True}}]

In [511]:
geoloc_MLG=MALAGA.aggregate(query_mlg)

geoquery_malaga=json.loads(dumps(geoloc_MLG))

In [512]:
df_malaga = pd.DataFrame(geoquery_malaga)

In [513]:
df_malaga.drop_duplicates(["latitud","longitud"],inplace=True)


In [514]:
malaga_agrupado=df_malaga.groupby(["categoria"])["distance"].mean()

dfMALAGA=pd.DataFrame(malaga_agrupado).reset_index()

dfMALAGA

,categoria,distance
0,Aeropuerto,9.192777
1,Disco,18.134031
2,Diseño,2.351238
3,Starbucks,21.139791
4,Vegan,11.766978


In [531]:
func.normalize(dfMALAGA,"distance")

dfMALAGA["normalizado"]=func.normalize(dfMALAGA,"distance")

dfMALAGA["weights"] = func.weights(dfMALAGA)

dfMALAGA

,categoria,distance,normalizado,weights
0,Aeropuerto,9.192777,0.364133,0.036413
1,Disco,18.134031,0.840022,0.168004
2,Diseño,2.351238,0.000000,0.000000
3,Starbucks,21.139791,1.000000,0.400000
4,Vegan,11.766978,0.501142,0.050114


In [532]:
dfMALAGA["ciudad"] = "Malaga"

In [533]:
dfMALAGA

,categoria,distance,normalizado,weights,ciudad
0,Aeropuerto,9.192777,0.364133,0.036413,Malaga
1,Disco,18.134031,0.840022,0.168004,Malaga
2,Diseño,2.351238,0.000000,0.000000,Malaga
3,Starbucks,21.139791,1.000000,0.400000,Malaga
4,Vegan,11.766978,0.501142,0.050114,Malaga


#### C. Barcelona:

In [534]:
BARCELONA = db.get_collection("BARCELONA") # Me conecto a la colección "BARCELONA".

BARCELONA

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROYECTO_3'), 'BARCELONA')

In [517]:
db.BARCELONA.create_index([("location", GEOSPHERE)]) # Creo el index 2dsphere.

'location_2dsphere'

In [518]:
query_bcn=[{
"$geoNear": {'near': [41.38781, 2.16920],
             'distanceField': 'distance',
             'maxDistance': 2000,
             'distanceMultiplier': 6371,
             'spherical'  : True}}]

In [519]:
geoloc_bcn=BARCELONA.aggregate(query_bcn)

geoquery_barcelona=json.loads(dumps(geoloc_bcn))

In [520]:
df_barcelona = pd.DataFrame(geoquery_barcelona)

df_barcelona.drop_duplicates(["latitud","longitud"],inplace=True)

In [521]:
df_barcelona.head()

barcelona_agrupado=df_barcelona.groupby(["categoria"])["distance"].mean()

In [522]:
dfBARCELONA=pd.DataFrame(barcelona_agrupado).reset_index()

dfBARCELONA

,categoria,distance
0,Aeropuerto,14.555272
1,Disco,1.887830
2,Diseño,2.539639
3,Starbucks,2.317484
4,Vegan,2.621971


In [523]:
func.normalize(dfBARCELONA,"distance")

dfBARCELONA["normalizado"]=func.normalize(dfBARCELONA,"distance")

dfBARCELONA


,categoria,distance,normalizado
0,Aeropuerto,14.555272,1.000000
1,Disco,1.887830,0.000000
2,Diseño,2.539639,0.051455
3,Starbucks,2.317484,0.033918
4,Vegan,2.621971,0.057955


In [525]:
dfBARCELONA["weights"] = func.weights(dfBARCELONA)

dfBARCELONA

,categoria,distance,normalizado,weights
0,Aeropuerto,14.555272,1.000000,0.100000
1,Disco,1.887830,0.000000,0.000000
2,Diseño,2.539639,0.051455,0.010291
3,Starbucks,2.317484,0.033918,0.013567
4,Vegan,2.621971,0.057955,0.005795


In [526]:
dfBARCELONA["ciudad"] = "Barcelona"

In [527]:
dfBARCELONA

,categoria,distance,normalizado,weights,ciudad
0,Aeropuerto,14.555272,1.000000,0.100000,Barcelona
1,Disco,1.887830,0.000000,0.000000,Barcelona
2,Diseño,2.539639,0.051455,0.010291,Barcelona
3,Starbucks,2.317484,0.033918,0.013567,Barcelona
4,Vegan,2.621971,0.057955,0.005795,Barcelona


### Concateno los tres dataframes agrupados por categorías de cada ciudad:

In [535]:
concatenado=func.concat_ciudades(dfMADRID,dfMALAGA,dfBARCELONA)

In [536]:
concatenado

,categoria,distance,normalizado,weights,ciudad
0,Disco,3.478146,0.000000,0.000000,Madrid
1,Diseño,3.664390,0.025704,0.005141,Madrid
2,Starbucks,3.912457,0.059939,0.023976,Madrid
3,Vegan,4.920965,0.199124,0.019912,Madrid
4,aeropuerto,10.723970,1.000000,0.100000,Madrid
0,Aeropuerto,9.192777,0.364133,0.036413,Malaga
1,Disco,18.134031,0.840022,0.168004,Malaga
2,Diseño,2.351238,0.000000,0.000000,Malaga
3,Starbucks,21.139791,1.000000,0.400000,Malaga
4,Vegan,11.766978,0.501142,0.050114,Malaga


In [537]:
final=concatenado.groupby(["ciudad"])["weights"].sum()

In [538]:
ranking=pd.DataFrame(final).reset_index()

In [539]:
ranking

,ciudad,weights
0,Barcelona,0.129654
1,Madrid,0.149029
2,Malaga,0.654532


In [577]:
ranking.to_csv("../data/rankingfinal.csv")

### A. Map MADRID:

In [540]:
madrid

{'type': 'Point', 'coordinates': ['40.44980', '-3.69164']}

In [541]:
lat=madrid["coordinates"][0]

In [542]:
long=madrid["coordinates"][1]

In [543]:
inicial_lat = float(lat)
inicial_long =float(long)

In [545]:
map_1 = Map(location = [inicial_lat,inicial_long], zoom_start = 15)

In [459]:
MADRID_df.categoria.unique()

array(['Diseño', 'Starbucks', 'aeropuerto', 'Disco', 'Vegan'],
      dtype=object)

In [575]:
for i,row in MADRID_df.iterrows():
    requirements = {"location": [row["latitud"], row["longitud"]], "tooltip": row["categoria"]}
    
    if row["categoria"] == "Diseño":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="laptop",
                     icon_color="black")
        
    elif row["categoria"] == "Starbucks":
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="coffee",
                     icon_color="black")
        
    elif row["categoria"] == "aeropuerto":
        icono = Icon(color = "orange",
                     prefix="fa",
                     icon="plane",
                     icon_color="black")
        
    elif row["categoria"] == "Disco":
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="glass",
                     icon_color="black")
    
    elif row["categoria"] == "Vegan":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="heart",
                     icon_color="black")
    
    mark = Marker(**requirements, icon=icono)
    mark.add_to(map_1)

In [576]:
map_1

### B. Map Málaga:

In [547]:
malaga

{'type': 'Point', 'coordinates': ['36.72048', '-4.42166']}

In [548]:
lat=malaga["coordinates"][0]

In [549]:
long=malaga["coordinates"][1]

In [550]:
inicial_lat2 = float(lat)
inicial_long2 =float(long)

In [551]:
map_2 = Map(location = [inicial_lat2,inicial_long2], zoom_start = 15)

In [552]:
MALAGA_df.categoria.unique()

array(['Diseño', 'Aeropuerto', 'Starbucks', 'Disco', 'Vegan'],
      dtype=object)

In [573]:
for i,row in MALAGA_df.iterrows():
    requirements = {"location": [row["latitud"], row["longitud"]], "tooltip": row["categoria"]}
    
    if row["categoria"] == "Diseño":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="laptop",
                     icon_color="black")
        
    elif row["categoria"] == "Aeropuerto":
        icono = Icon(color = "orange",
                     prefix="fa",
                     icon="plane",
                     icon_color="black")
        
    elif row["categoria"] == "Starbucks":
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="coffee",
                     icon_color="black")
        
    elif row["categoria"] == "Disco":
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="glass",
                     icon_color="black")
    
    elif row["categoria"] == "Vegan":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="heart",
                     icon_color="black")
    
    mark = Marker(**requirements, icon=icono)
    mark.add_to(map_2)

In [574]:
map_2

### C. Map Barcelona:

In [555]:
barcelona

{'type': 'Point', 'coordinates': ['41.38781', '2.16920']}

In [556]:
lat=barcelona["coordinates"][0]

In [557]:
long=barcelona["coordinates"][1]

In [558]:
inicial_lat3 = float(lat)
inicial_long3 =float(long)

In [567]:
map_3 = Map(location = [inicial_lat3,inicial_long3], zoom_start = 15)

In [568]:
BARCELONA_df.categoria.unique()

array(['Diseño', 'Aeropuerto', 'Starbucks', 'Disco', 'Vegan'],
      dtype=object)

In [571]:
for i,row in BARCELONA_df.iterrows():
    requirements = {"location": [row["latitud"], row["longitud"]], "tooltip": row["categoria"]}
    
    if row["categoria"] == "Diseño":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="laptop",
                     icon_color="black")
        
    elif row["categoria"] == "Aeropuerto":
        icono = Icon(color = "orange",
                     prefix="fa",
                     icon="plane",
                     icon_color="black")
        
    elif row["categoria"] == "Starbucks":
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="coffee",
                     icon_color="black")
        
    elif row["categoria"] == "Disco":
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="glass",
                     icon_color="black")
    
    elif row["categoria"] == "Vegan":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="heart",
                     icon_color="black")
    
    mark = Marker(**requirements, icon=icono)
    mark.add_to(map_3)

In [572]:
map_3